In [25]:
# If not installed, run this cell. It is needed for access to GEE
%pip install ee

^C


In [26]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jvg')

In [ ]:
# If not installed, run this cell. It helps visualization of data
%pip install geemap
%pip show geemap

In [27]:
import geemap
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [28]:
# Access an image asset from my account
guate_bdry = ee.FeatureCollection('users/jvg/guatemala_buffer')
guate_geom = guate_bdry.geometry();
Map.centerObject(guate_geom);
Map.addLayer(guate_bdry, {"color": 'red'}, 'Guatemala Boundary');

In [29]:
clay = ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02')
clay_clipped = clay.clip(guate_bdry)
Map.addLayer(clay_clipped, {}, 'clay_clipped')

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ee: filename=ee-0.2-py3-none-any.whl size=3676 sha256=cc9cf20a899e08c73dffa030f0706930a94a9bafe5a095fb9165905b74034ab3
  Stored in directory: c:\users\jvalenciag\appdata\local\pip\cache\wheels\0d\6c\c0\f17fb105d1ad88aa5aff8067de0ebf7e141ea81f0df52535b9
Successfully built ee


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [30]:
# Let's import other libraries
import pandas as pd

: 